# core

> Fill in a module description here

In [ ]:
#| default_exp pheno

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional

import jax
import jax.numpy as jnp
from jaxtyping import Array, Float

from chewc.pop import Population
from chewc.sp import SimParam
from chewc.trait import _calculate_genetic_params

# %% ../nbs/04_phenotype.ipynb 4
def set_pheno(
    key: jax.random.PRNGKey,
    pop: Population,
    sim_param: SimParam,
    h2: Float[Array, "nTraits"],
    cor_e: Optional[Float[Array, "nTraits nTraits"]] = None,
) -> Population:
    """
    Sets the phenotype of a population based on genetic values and environmental noise.

    This function follows the AlphaSimR methodology by first calculating the total
    genetic value (GV) for each individual. It then uses the provided heritability (h2)
    to determine the amount of environmental variance, and finally adds this
    environmental noise to the genetic values to produce the final phenotypes.

    Args:
        key: A JAX random key used for generating the environmental noise.
        pop: The population for which to set the phenotype.
        sim_param: The simulation parameters, which include the trait definitions.
        h2: A JAX array with the narrow-sense heritability for each trait.
        cor_e: An optional environmental correlation matrix. If not provided,
               it is assumed that the environmental effects are uncorrelated.

    Returns:
        A new Population object with the `pheno` attribute updated.
    """
    n_traits = sim_param.n_traits
    if cor_e is None:
        cor_e = jnp.identity(n_traits)

    # 1. Calculate the genetic values for each trait
    gvs = []
    for trait in sim_param.traits:
        genetic_params = _calculate_genetic_params(trait, pop, sim_param)
        gvs.append(genetic_params["gv"])
    gvs = jnp.stack(gvs, axis=1)

    # 2. Calculate the environmental noise
    var_g = jnp.var(gvs, axis=0)
    var_e = (var_g / h2) - var_g
    cov_e = jnp.diag(jnp.sqrt(var_e)) @ cor_e @ jnp.diag(jnp.sqrt(var_e))

    environmental_noise = jax.random.multivariate_normal(
        key, jnp.zeros(n_traits), cov_e, (pop.nInd,)
    )

    # 3. Calculate the final phenotypes
    pheno = gvs + environmental_noise

    return pop.replace(pheno=pheno)








In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()